# Creating a custom Wind Tunnel scenario

In this notebook, we will briefly explain how to create a custom scenario to run Wind Tunnel simulations. 

The fundamental blocks required to create a scenario:

- Choose the simulators you want to use among the ones available via the **Inductiva API**;
- Template the input files for the simulators;
- Construct a Python class from the base class `Scenario` withing **Inductiva API**.


### Choosing a simulator

For this scenario, we will use the OpenFOAM simulator which is accessible with **Inductiva API**. The simulator is called `OpenFOAM` and it is available in the `inductiva.simulators` module. Look [at the documentation](https://github.com/inductiva/inductiva/wiki/OpenFOAM) for more information about the simulator.

### Templating the input files

Starting from previously prepared simulation configuration files, we can select a few parameters which we allow to be modified and the rest of the file will be used as is. The parameters to be modified are templated in the input files and will be the arguments passed in the class methods that allow to modify the scenario.

Each parameter we want to modify is templated in the input file using the following syntax `{{ parameter_name }}`. In the case of the OpenFOAM, let's show an example with the `controlDict` file:

<div>
<img src="../assets/template_example.png" alt="Template Example" width="300">
</div>

Then, we need to append the suffix `.jinja` to each file that contains a template parameter. In this case, the file will be called `controlDict.jinja`.

For this scenario, we template the following parameters:
- `flow_velocity`: the velocity of the flow in the wind tunnel;
- `domain`: the domain geometry of the wind tunnel (assuming a rectangular box shape);
- `object_path`: the object we want to insert inside the domain;
- `num_iterations`: the number of iterations the solver will run to find the steady-state;
- `resolution`: a parameter to control the resolution of the mesh.

We want to highlight that these variables can be anything you want: floats, lists, dictionaries. For example, we will assume that the `domain` is a dict given as `domain={"x": [0, 10], "y": [-5, 5], "z": [0, 8]}`. Then, we modify the `blockMeshDict.jinja` file as follows:

<div>
<img src="../assets/domain_template.png" alt="Template Example" width="400">
</div>

The scenario is only limited by your imagination!

### Construct the custom Scenario class

As mentioned we will start from the base class `Scenario` in Inductiva API. The class will contain the following methods:
-   `__init__`: Initializes the scenario with the parameters of the physical model;
-   `simulate`: Launch the simulation of the physical model and adjust the simulation parameters.

All templated parameters need to be passed to the base class attribute `params` in order to be substituted in the template files. Keep the same naming convention as in the template files. 

Furthermore, the `simulate` method inherits three arguments from the base class: 
- the `simulator`: simulator to run the simulation scenario.
- the `machine_group`: the machines where the simulation will run.
- the `storage_dir`: the directory on the Cloud to store the simulation.

Then, since we want users to pass their own object, we will use `add_extra_input_files` method to insert the object in the right place. For the case of the OpenFOAM is in the folder `constant/triSurface`. Then, the method will be called automatically when the simulation is launched.


Let's start implementing the class!

In [ ]:
import os
import inductiva

class WindTunnel(inductiva.scenarios.Scenario):
    """Implementation of a custom WindTunnel scenario."""

    # Class attributes that are used within the base scenario class.
    valid_simulators = [inductiva.simulators.OpenFOAM]
    template_files_dir = "templates"

    def __init__(self,
                 flow_velocity = None,
                 domain = None):
        """Initialise the scenario.

        Args:
            flow_velocity: List of floats describing the direction of the flow.
            domain: Dictionary describing the domain of the simulation.

        """

        self.params["flow_velocity"] = flow_velocity
        self.params["domain"] = domain

    def simulate(
        self,
        simulator = inductiva.simulators.OpenFOAM(),
        machine_group = None,
        storage_dir = "",
        object_path = None,
        num_iterations = 100,
        resolution = 2
    ):
        """Run the simulation scenario.

        Args:
            object_path: Path to the object to be inside the WindTunnel.
            num_iterations: Number of iterations to simulate.
            resolution: Integer controlling the resolution of the simulation.
        """

        self.object_path = object_path
        self.params["num_iterations"] = num_iterations
        self.params["resolution"] = resolution

        commands = self.get_commands()
        task = super().simulate(simulator,
                                machine_group=machine_group,
                                storage_dir=storage_dir,
                                commands=commands)

        return task

    def pre_simulate_hook(self):
        """Set pre-simulation parameters."""

        output_path = os.path.join("constant", "triSurface", "object.obj")
        self.add_extra_input_files(self.object_path, output_path)


